In [1]:
# transform the court name
def courtName(str):
    courtName = {'南投':'NTDV', '臺中':'TCDV', '台北':'TPDV', '臺南':'TNDV', '臺東':'TTDV', '嘉義':'CYDV',\
                 '基隆':'KLDV', '士林':'SLDV', '宜蘭':'ILDV', '屏東':'PTDV', '彰化':'CHDV', '新北':'PCDV',\
                 '新竹':'SCDV', '桃園':'TYDV', '澎湖':'PHDV', '花蓮':'HLDV', '苗栗':'MLDV', '連江':'LCDV',\
                 '金門':'KMDV', '雲林':'ULDV', '高雄':'KSDV'}
    return courtName[str]

In [2]:
# transform the jud_case to url encode
def jud_case(str):
    import urllib
    jud_case = {'':str}
    jud_case_encode = urllib.urlencode(jud_case)[1:]
    return jud_case_encode

In [3]:
# read target from list
# data format: 9960-臺中_90_中簡_2622.txt
def getTarget(str):
    import codecs
    f = codecs.open(str,'r','utf-8')
    fcontent = f.read().encode('utf8')
    raw = fcontent.split('\r\n')
    ary = []
    for row in raw:
        sep = row.strip().split('-')[1].split(".")[0].split("_")
        # aggregate the search data to a dic
        encoData = {'courtFullName':courtName(sep[0]), 'jud_year':sep[1], 'jud_case':jud_case(sep[2]), 'jud_no':sep[3]}
        # this one for file saving
        rawData = {'courtFullName':sep[0], 'jud_year':sep[1], 'jud_case':sep[2], 'jud_no':sep[3]}
        # aggregate to ary
        ary.append([encoData, rawData])
    f.close()
    return ary

In [4]:
# retrieve the x value and return cookie
def getCookie(court, year, case, no):
    import requests
    from bs4 import BeautifulSoup
    # insert specify attributes for each case
    url = 'http://fyjud.lawbank.com.tw/Receive2.aspx?courtFullName='+court+\
           '&v_court=&v_sys=&jud_year='+year+\
           '&jud_case='+case+\
           '&jud_no='+no+\
           '&jud_title=&jud_jmain=&keyword=&sdate=&edate=&file=&page=&id=&searchkw=&jcatagory=2'+\
           '&switchFrom=&issimple=-1'+\
           '&jminmoney=&jmaxmoney=&jminyear=&jmaxyear=&txtjudge=&txtlawyer=&lc1a=&lc1b=&lc1c='
    res = requests.get(url)
    soup = BeautifulSoup(res.text)
    # select the right part we need
    j = soup.select('a')[0]['onclick'].split("'")[1]
    # append it after j=
    x = 'j='+j
    # setCookie
    cookie = {'y':'',\
              'ASP.NET_SessionId':'zjafvq45bo1r5zvxjq0nyw55',\
              'JubFrm-pagebox':'%5EcourtFullName%3D'+court+\
                               '%60%5Ejud_year%3D'+year+\
                               '%5Ejud_case%3D'+case+\
                               '%5Ejud_no%3D'+no+\
                               '%5Ejcatagory%3D2%5Eissimple%3D-1',\
              '_ga':'GA1.3.473815266.1434356225',\
              '_gat':'1',\
              'x':x}
    return cookie

In [5]:
# compare and retrieve the link/cookie_y of duplicate one
def targetLinkY(ck):
    import requests
    from bs4 import BeautifulSoup
    res = requests.get('http://fyjud.lawbank.com.tw/'+'listcontent5.aspx', cookies = ck)
    # create a list for choosing the right one by comparing file size
    compare = []
    soup = BeautifulSoup(res.text)
    for each in soup.select('td[nowrap]'):
        # append the file size to list
        if each.text != '':
            # change the last int 1 to 2 if your data is like '花蓮_89_訴更(一)_5.txt'
            compare.append(int(each.text[:-2].strip().split('(')[1]))
        # append -1 if the tag is empty
        else:
            compare.append(-1)
    # find the index of the max value
    index = compare.index(max(compare))
    # parse the link
    contentLink = soup.select('td[nowrap]')[index].select('a')[0]['href']
    # parse cookie_y value
    y = 'p='+soup.select('td[nowrap]')[index].select('a')[0]['onclick'].split("'")[1]
    return {'contentLink':contentLink, 'y':y}

In [6]:
def writeContent(li):
    import requests
    from bs4 import BeautifulSoup
    cookie['y'] = linkY['y']
    contentUrl = 'http://fyjud.lawbank.com.tw/'+linkY['contentLink']
    res = requests.get(contentUrl, cookies = cookie)       
    res_soup = BeautifulSoup(res.text)
    path='%s_%s_%s_%s.txt'%(li[1]['courtFullName'], li[1]['jud_year'], li[1]['jud_case'], li[1]['jud_no'])
    f = open(path.decode('utf-8'),'w')
    for pre in res_soup.select('pre'):      
        f.write(pre.text.encode('utf-8')+'\n')
    f.close()
        

In [14]:
# main
list = getTarget('missing.txt')
i = 1
#j = 1718
for target in list:
    cookie = getCookie(target[0]['courtFullName'], target[0]['jud_year'], target[0]['jud_case'], target[0]['jud_no'])
    linkY = targetLinkY(cookie)
    writeContent(target)
    print i, target[1]['courtFullName'], target[1]['jud_year'], target[1]['jud_case'], target[1]['jud_no'], "DONE"
    i = i + 1

1 士林 93 湖簡 387 DONE
2 新北 93 保險 14 DONE


## Note
- edit LinkY to linkY <-- Done
- def writeContent() -> def writeContent(list), also the args <-- Done
- skip 227, 1040, 1041 <-- Done

## Find the missing file

In [8]:
fAll = open('duplicated_list_oringinal_for_get_missing.txt', 'r')
fRes = open('result_list.txt', 'r')
#fMis = open('missing_list.txt', 'w')
fAllContent = fAll.read().split('\n')
fResContent = fRes.read().split('\n')
for line in fAllContent:
    if line not in fResContent:
        print line
fAll.close()
fRes.close()
#fMis.close()

## Test code below

In [78]:
a = ['a','b','c']
b = ['a','d']

for i in a:
    if i not in b:
        print i

b
c


In [23]:
cookie = getCookie('TPDV', '90', '%E5%BA%97%E5%B0%8F', '297')
print cookie

{'ASP.NET_SessionId': 'zjafvq45bo1r5zvxjq0nyw55', 'JubFrm-pagebox': '%5EcourtFullName%3DTPDV%60%5Ejud_year%3D90%5Ejud_case%3D%E5%BA%97%E5%B0%8F%5Ejud_no%3D297%5Ejcatagory%3D2%5Eissimple%3D-1', '_ga': 'GA1.3.473815266.1434356225', 'y': '', 'x': u'j=ExKZIgGGiQD19KUxjSulmQb58Xq9tzxh/6k2CZDMXMbqn+pyISjsF42Gyv8r7WXOK2TSUk8PjqCjFgL8iLOg+iNkAojTlLxxWDw0QavV6OyoT7gSIGUwE2HJ47ggCB/3IZ/ze2onHPHbrig7DIXsGf/LKE1ffRqwJWbkFYC5v/dZGQDZG2p0ebbypK98JXfGaS1xhZcx5F3BuxM9jbPUy5VEIyeZH9KKjapjEXVcjOh3MX5ya4aG75C7j2NOFzExMEKptNhMJMau08NOiY2m8g==', '_gat': '1'}


In [52]:
linkY = targetLinkY(cookie)
print linkY

{'contentLink': u'content3.aspx?p=NFLNW3o2iZnLciVoeEtcUHAcgcEWaOO3rtBQxD%2bfl4A%3d', 'y': u'p=wbWCgBlMA2QgJSEANuPzEsMcWQWY8JyOOlZiaeJB5cIz2GfzriHtlEpHACPm9PwA'}


In [ ]:
import pandas as pd
df = pd.DataFrame(list[0])
df

In [17]:
# -*- coding: utf8 -*-
encoded = '\xe4\xbb\x8a\xe5\xa4\xa9\xe5\xa4\xa9\xe6\xb0\xa3\xe7\x9c\x9f\xe5\xa5\xbd'
msg = encoded.decode('utf8')
print msg

# -*- coding: utf8 -*-
msg = u'今天天氣真好'
encoded = msg.encode('utf8')
print encoded
print repr(encoded)

print (msg == encoded)

今天天氣真好
今天天氣真好
'\xe4\xbb\x8a\xe5\xa4\xa9\xe5\xa4\xa9\xe6\xb0\xa3\xe7\x9c\x9f\xe5\xa5\xbd'
False


C:\Python27\lib\site-packages\IPython\kernel\__main__.py:12: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
